In [ ]:
"""
Purpose: To check on the status of the soma finder


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

In [193]:
import minfig
du = reload(du)
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-22 07:06:30,918 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 07:06:30,920 - settings - Setting database.user to celiib
INFO - 2020-11-22 07:06:30,921 - settings - Setting database.password to newceliipass
INFO - 2020-11-22 07:06:30,923 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 07:06:30,924 - settings - Setting database.user to celiib
INFO - 2020-11-22 07:06:30,926 - settings - Setting database.password to newceliipass
INFO - 2020-11-22 07:06:30,931 - settings - Setting enable_python_native_blobs to True


Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2020-11-22 07:06:31,161 - settings - Setting enable_python_native_blobs to True


# Inspecting the Decimation

In [143]:
(schema.jobs & "table_name='__decimation'").delete()

In [88]:
len(minnie.Mesh() & minnie.SegToDecimateFromNuclei())

114306

In [142]:
decimation_version = 0
decimation_ratio = 0.25
len(minnie.Decimation().proj(decimation_version='version') & "decimation_version=" + str(decimation_version) &
                   f"decimation_ratio={decimation_ratio}" & minnie.SegToDecimateFromNuclei() ) - 114201

1

# Inspecting the Soma Extraction

In [164]:
(schema.jobs & 'table_name="__baylor_segment_centroid"').delete()

In [96]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000

key_source =  ((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                        "decimation_version=" + str(decimation_version) &
                   f"decimation_ratio={decimation_ratio}" & minnie.SegToDecimateFromNuclei())
len(key_source)

87437

In [167]:
len(key_source) - len(key_source.proj() & (minnie.BaylorSegmentCentroid()).proj())

356

In [ ]:
len(minnie.BaylorSegmentCentroid() & "multiplicity>0")/len(minnie.BaylorSegmentCentroid())*len(key_source)

# Chccking the current progress of Soma Extraction

In [166]:
my_key = (minnie.BaylorSegmentCentroid() & "segment_id=864691134026921572").fetch1()
my_key

{'segment_id': 864691134026921572,
 'decimation_version': 0,
 'decimation_ratio': Decimal('0.25'),
 'soma_index': 0,
 'centroid_x': None,
 'centroid_y': None,
 'centroid_z': None,
 'n_vertices': 0,
 'n_faces': 0,
 'soma_vertices': None,
 'soma_faces': None,
 'multiplicity': 0,
 'sdf': None,
 'max_side_ratio': None,
 'bbox_volume_ratio': None,
 'max_hole_length': None,
 'run_time': 51.24646282196045}

In [ ]:
"""



"""

In [192]:
minnie.Decimation.describe()

# Decimated meshes
-> minnie.Mesh
-> minnie.DecimationConfig
---
n_vertices           : bigint                       
n_faces              : bigint                       
mesh                 : <decimated_mesh>             # in-place path to the hdf5 (decimated) mesh file
INDEX (mesh)



'# Decimated meshes\n-> minnie.Mesh\n-> minnie.DecimationConfig\n---\nn_vertices           : bigint                       \nn_faces              : bigint                       \nmesh                 : <decimated_mesh>             # in-place path to the hdf5 (decimated) mesh file\nINDEX (mesh)\n'

In [195]:
minnie.BaylorSegmentCentroidExternal & "segment_id=864691134026921572"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,mesh datajoint adapter to get the somas mesh objects,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)
864691134026921572,0,0.25,0,None,None,None,0,0,=BLOB=,0,nan,nan,nan,nan,51.24646282196045


In [228]:
len(minnie.BaylorSegmentCentroid())

91832

In [254]:
len(minnie.BaylorSegmentCentroidExternal())

91832

In [178]:
p = Path('/mnt/dj-stor01/platinum/minnie65/02/somas/')

In [180]:
for f in p.iterdir():
    f.unlink()

In [239]:
no_soma_seg_id = (minnie.Decimation() & (minnie.BaylorSegmentCentroidExternal & "multiplicity=0").proj() & "n_vertices>50000").fetch("segment_id")

In [260]:
curr_seg_id = no_soma_seg_id[200]

In [261]:
du.plot_decimated_mesh_with_somas(curr_seg_id)

INFO - 2020-11-22 08:54:28,429 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 08:54:28,780 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 08:54:30,412 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 08:54:30,759 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 08:54:30,770 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 08:54:31,108 - settings - Setting enable_python_native_blobs to True
/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [262]:
curr_mesh = du.fetch_segment_id_mesh(curr_seg_id)

INFO - 2020-11-22 08:54:31,201 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 08:54:31,553 - settings - Setting enable_python_native_blobs to True


In [263]:
import soma_extraction_utils as sm
soma_data = sm.extract_soma_center(segment_id=curr_seg_id,
                      current_mesh_verts=curr_mesh.vertices,
                      current_mesh_faces=curr_mesh.faces)

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 5000.0 
large_mesh_threshold_inner = 5000.0 
soma_size_threshold = 937.5 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 4775 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_26909.off -o /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_26909_fill_holes.off -s /notebooks/Platinum_Datajoint/Soma_Extraction/temp/fill_holes_644082.mls

---- meshlab output -----
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
Current Plugins Dir is: /meshlab/src/distrib/plugins 
Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files
Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or 

WARNING - 2020-11-22 08:54:57,631 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,632 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,649 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,650 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,783 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,785 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,807 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,858 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,861 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,936 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,938 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,950 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:57,963 - base - face_normals all zero, ignoring!
WARNING - 20

WARNING - 2020-11-22 08:54:59,755 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,758 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,765 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,768 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,775 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,778 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,803 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,809 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,861 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,874 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,927 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,930 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:54:59,937 - base - face_normals all zero, ignoring!
WARNING - 20

Removing the following inside neurons: [<trimesh.Trimesh(vertices.shape=(2853, 3), faces.shape=(6331, 3))>, <trimesh.Trimesh(vertices.shape=(2721, 3), faces.shape=(6525, 3))>, <trimesh.Trimesh(vertices.shape=(1954, 3), faces.shape=(4622, 3))>, <trimesh.Trimesh(vertices.shape=(1723, 3), faces.shape=(2712, 3))>, <trimesh.Trimesh(vertices.shape=(1686, 3), faces.shape=(2629, 3))>, <trimesh.Trimesh(vertices.shape=(1623, 3), faces.shape=(2497, 3))>, <trimesh.Trimesh(vertices.shape=(1613, 3), faces.shape=(2477, 3))>, <trimesh.Trimesh(vertices.shape=(1502, 3), faces.shape=(3854, 3))>, <trimesh.Trimesh(vertices.shape=(1178, 3), faces.shape=(2712, 3))>, <trimesh.Trimesh(vertices.shape=(1122, 3), faces.shape=(2583, 3))>, <trimesh.Trimesh(vertices.shape=(998, 3), faces.shape=(2400, 3))>, <trimesh.Trimesh(vertices.shape=(894, 3), faces.shape=(2047, 3))>, <trimesh.Trimesh(vertices.shape=(824, 3), faces.shape=(1942, 3))>, <trimesh.Trimesh(vertices.shape=(757, 3), faces.shape=(1117, 3))>, <trimesh.Tri

WARNING - 2020-11-22 08:55:08,930 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,044 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,045 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,143 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,146 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,152 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,162 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,172 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,178 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,179 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,189 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,190 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:09,193 - base - face_normals all zero, ignoring!
WARNING - 20

Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(20810, 3), faces.shape=(41357, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(20810, 3), faces.shape=(41357, 3))>
remove_inside_pieces requested 
xvfb-run -n 2268 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_83713.off -o /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_83713_fill_holes.off -s /notebooks/Platinum_Datajoint/Soma_Extraction/temp/fill_holes_36223.mls
removed temporary input file: /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_83713.off
removed temporary output file: /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_83713_fill_holes.off
/notebooks/Platinum_Datajoint/Soma_Extraction/temp/fill_holes_36223.mls is being deleted....
xvfb-run -n 4200 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_18863.off -o /notebooks/Platinum_Dat

WARNING - 2020-11-22 08:55:41,789 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:41,902 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:41,903 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:41,998 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:42,001 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:42,007 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:42,017 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:42,026 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:42,032 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:42,033 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:42,042 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:42,043 - base - face_normals all zero, ignoring!
WARNING - 2020-11-22 08:55:42,047 - base - face_normals all zero, ignoring!
WARNING - 20

# total split meshes = 30
viable_meshes = [0]
There were 29 pieces found after size threshold
# of soma containing seperate meshes = 1
meshes with somas = {0: [0]}


----Working on soma-containing mesh piece 0----
current_soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(1950, 3), faces.shape=(3711, 3))>]
current_mesh = <trimesh.Trimesh(vertices.shape=(97900, 3), faces.shape=(195455, 3))>

inside Soma subtraction
mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(32884, 3), faces.shape=(65564, 3))>, <trimesh.Trimesh(vertices.shape=(26820, 3), faces.shape=(53512, 3))>, <trimesh.Trimesh(vertices.shape=(7562, 3), faces.shape=(15047, 3))>, <trimesh.Trimesh(vertices.shape=(5781, 3), faces.shape=(11539, 3))>, <trimesh.Trimesh(vertices.shape=(5683, 3), faces.shape=(11326, 3))>, <trimesh.Trimesh(vertices.shape=(1981, 3), faces.shape=(3936, 3))>, <trimesh.Trimesh(vertices.shape=(1630, 3), faces.shape=(3235, 3))>, <trimesh.Trimesh(vertices.shape=(


Largest hole before segmentation = 53012.86141472555, after = 1164182.7847314859,
ratio = 21.96038383259401, difference = 1111169.9233167602


/meshAfterParty/trimesh_utils.py:996: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  curr_p_verts = np.unique(main_mesh.faces[curr_p_faces].ravel())
/meshAfterParty/networkx_utils.py:511: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cycles_list_array = np.array(cycles_list)


In [264]:
nviz.plot_objects(meshes=soma_data[0])

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))
